## スコアリングフェーズにおけるデータ処理（課題把握編）

ローン審査データを使って<b>モデリング段階のデータ処理をおさらいし、</b><br>その後、スコアリング段階のデータ処理で必要となるテクニックを学びましょう。

In [ ]:
# import sample data: Loan screening data for classification 
import pandas as pd

df = pd.read_csv('./data/av_loan_u6lujuX_CVtuZ9i.csv',header=0)
X  = df.iloc[:,:-1]           # 最終列以前を特徴量X
ID = X.iloc[:,[0]]            # 最初列がPK（Loan_ID）なのでID情報としてセット
X  = X.drop('Loan_ID',axis=1) # 1列目(Loan_ID)は特徴量ベクトルから削除
y  = df.iloc[:,-1]            # 最終列を正解データ

# check the shape
print('--------------------------------------')
print('Raw shape: (%i,%i)' %df.shape)
print('X shape: (%i,%i)' %X.shape)

# converting stirng to number(binary flag)
# ローン審査でNOとなったサンプルを1（正例）として変換
class_mapping = {'N':1, 'Y':0}
y = y.map(class_mapping)
print('---------------------------------------')
print(y.value_counts())
print('---------------------------------------')
print(ID.join(X).join(y).dtypes)
ID.join(X).join(y).head()

モデリング段階の前処理として、まずはone-hotエンコーディングを実施します。

In [ ]:
ohe_columns = ['Dependents',
               'Gender',
               'Married',
               'Education',
               'Self_Employed',
               'Property_Area']
X_ohe = pd.get_dummies(X,
                       dummy_na=True,
                       columns=ohe_columns)
print('X_ohe shape:(%i,%i)' % X_ohe.shape)
X_ohe.head()

続いて、連続変数の欠損を平均値で置き換えます。

In [ ]:
from sklearn.preprocessing import Imputer

# 欠損値NaNを平均値(mean)で置換
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp.fit(X_ohe)

# 学習済みImputerを適用しX_newの欠損値を置換
X_ohe_columns = X_ohe.columns.values
X_ohe = pd.DataFrame(imp.transform(X_ohe), columns=X_ohe_columns)

# 結果表示
X_ohe.head()

最後に、RFEによる特徴量選択を実施します。

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier

selector = RFE(RandomForestClassifier(random_state=1),
               n_features_to_select=10,
               step=.05)

selector.fit(X_ohe,y)

X_fin = pd.DataFrame(selector.transform(X_ohe),
                     columns=X_ohe_columns[selector.support_])

print('X_fin shape:(%i,%i)' % X_fin.shape)
X_fin.head()

ここまでがモデリング段階でのデータ加工でした。さて未知のデータ（スコア用データ）に対しては、<b>上記10次元の特徴量をこの並びの通りに変換しなくてはいけません</b>。なぜなら、本データを学習したモデルは、0次元目から9次元目まで当該データ列が並んでいることを前提に予測値が計算されるからです。それでは、<b>いよいよスコア用データへの処理</b>です。

In [ ]:
# import sample data for classificatio
df_s = pd.read_csv('./data/av_loan_test_Y3wMUE5_7gLdaTN.csv', header=0)
ID_s = df_s.iloc[:,[0]]            # 第0列はPK（Loan_ID）なのでIDとしてセット
X_s  = df_s.drop('Loan_ID',axis=1) # Loan_IDはID情報なので特徴ベクトルから削除

# check the shape
print('Raw shape: (%i,%i)' %df_s.shape)
print('X shape: (%i,%i)' %X_s.shape)
print('-------------------------------')
print(X_s.dtypes)

まずはモデリング段階と同様、one-hotエンコーディングを実施します。

In [ ]:
X_ohe_s = pd.get_dummies(X_s,
                         dummy_na=True,
                         columns=ohe_columns)
print('X_ohe_s shape:(%i,%i)' % X_ohe_s.shape)
X_ohe_s.head()

one-hotエンコーディング後のスコアリングデータの特徴量次元は26次元とモデリング時点（26次元）と同じですが、もう少し厳密に特徴量リストを比較してみます。徴量リストの集合の差を見てみましょう。

In [ ]:
cols_model = set(X_ohe.columns.values)
cols_score = set(X_ohe_s.columns.values)

# モデルにはあったスコアにはないデータ項目
diff1 = cols_model - cols_score
print('Modelのみ:%s' % diff1)

# スコアにはあるがモデルになかったデータ項目
diff2 = cols_score - cols_model
print('Scoreのみ:%s' % diff2)

実はこのスコアデータ、以下２つの細工がされたデータです。
1. Gender変数に"Unknown"という項目を新しく追加
2. Dependents変数の"3+"という項目を除外（残された値は、0,1,2の3つのカテゴリ値）

上記のため、モデルのみに存在のダミー変数「Dependent_3+」、スコアのみに存在のダミー変数「Gender_Unknow」が出現しました。残りの差異はPythonがDependentsをObject型と判断したか（モデルデータ時点）、float64型と判断したか（スコアデータ時点）の違いに起因したものです。まとめると、スコアデータのone-hotエンコーディング処理の結果は、以下のような不整合が発生し得るということです。
1. モデルデータにないカラムが生成される可能性（Gender_Unknown)
1. モデルデータにあったカラムが消える可能性（Dependents_3+）
1. データ型の違いが理由で①/②が生じる可能性